## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import json
import os
import numpy as np

from dotenv import load_dotenv
load_dotenv()

# Google_API_KEY = os.getenv("g_key")
Google_API_KEY = os.getenv("g_key")

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=os.environ["g_key"])

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,qaanaaq,GL,77.4840,-69.3632,44.56,53,4,4.72,clear sky
1,1,morondava,MG,-20.2833,44.2833,70.16,76,0,9.60,clear sky
2,2,albany,US,42.6001,-73.9662,89.33,70,64,1.01,broken clouds
3,3,cabo san lucas,MX,22.8909,-109.9124,93.61,72,75,8.05,broken clouds
4,4,mataura,NZ,-46.1927,168.8643,38.48,76,9,7.52,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
#3 . 
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,albany,US,42.6001,-73.9662,89.33,70,64,1.01,broken clouds
10,10,half moon bay,US,37.4636,-122.4286,83.57,61,75,21.85,broken clouds
12,12,vaini,TO,-21.2000,-175.2000,80.76,89,20,8.05,few clouds
13,13,paramirim,BR,-13.4425,-42.2389,76.86,50,22,10.18,few clouds
22,22,larnaca,CY,34.9167,33.6292,78.67,72,0,8.05,clear sky
24,24,atuona,PF,-9.8000,-139.0333,76.91,70,92,18.23,overcast clouds
32,32,paracuru,BR,-3.4100,-39.0306,80.11,80,10,14.45,clear sky
33,33,yuli,NG,9.6989,10.2744,75.18,90,100,5.70,overcast clouds
34,34,andros town,BS,24.7000,-77.7667,84.16,69,17,15.32,few clouds
40,40,ishigaki,JP,24.3448,124.1572,86.13,79,75,6.91,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,albany,US,89.33,broken clouds,42.6001,-73.9662,
10,half moon bay,US,83.57,broken clouds,37.4636,-122.4286,
12,vaini,TO,80.76,few clouds,-21.2000,-175.2000,
13,paramirim,BR,76.86,few clouds,-13.4425,-42.2389,
22,larnaca,CY,78.67,clear sky,34.9167,33.6292,
24,atuona,PF,76.91,overcast clouds,-9.8000,-139.0333,
32,paracuru,BR,80.11,clear sky,-3.4100,-39.0306,
33,yuli,NG,75.18,overcast clouds,9.6989,10.2744,
34,andros town,BS,84.16,few clouds,24.7000,-77.7667,
40,ishigaki,JP,86.13,broken clouds,24.3448,124.1572,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": Google_API_KEY
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,half moon bay,US,83.57,broken clouds,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
12,vaini,TO,80.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
13,paramirim,BR,76.86,few clouds,-13.4425,-42.2389,Pousada Rio Pequeno
22,larnaca,CY,78.67,clear sky,34.9167,33.6292,Flamingo Beach Hotel
24,atuona,PF,76.91,overcast clouds,-9.8000,-139.0333,Villa Enata


In [10]:
# 8a. Create the output File (CSV)
 
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,half moon bay,US,83.57,broken clouds,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
12,vaini,TO,80.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
13,paramirim,BR,76.86,few clouds,-13.4425,-42.2389,Pousada Rio Pequeno
22,larnaca,CY,78.67,clear sky,34.9167,33.6292,Flamingo Beach Hotel
24,atuona,PF,76.91,overcast clouds,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
546,gushi,CN,85.42,overcast clouds,34.6425,109.5843,Wansheng Yuan Hotel
547,fare,PF,76.89,broken clouds,-16.7000,-151.0167,Maitai Lapita Village Huahine
550,limbang,MY,77.11,broken clouds,4.7500,115.0000,Purnama Hotel
551,korla,CN,81.63,clear sky,41.7597,86.1469,Gulisitan Hotel


In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[['Lat', 'Lng']]

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))